In [42]:
import pandas as pd
from warnings import filterwarnings
import re
import textwrap
filterwarnings("ignore")

In [2]:
def overview(df , count = True , just_null = False , just_object = False):
    import pandas as pd
    
    print(f"  Number of Rows  = {df.shape[0]:,}")
    print(f"Number of Columns = {df.shape[1]:,}")

    table = pd.DataFrame([])
    table["Missing Percentage"] = (df.isna().sum().to_frame() / df.shape[0] * 100).round(1)
    table["Missing Count"] = df.isna().sum()
    table["Data Type"] = df.dtypes
    table.index.name = "Column"
    
    table = table.sort_values(by = ["Missing Count" , "Data Type"] , ascending = [False , False])
    
    if just_object == True:
        table = table.loc[table["Data Type"] == "object"]
    if just_null == True:
        table = table.loc[table["Missing Count"] > 0] 
    if count == False:
        table = table.drop("Missing Count" , axis = 1)
    else:
        table["Missing Count"] = table["Missing Count"].apply(lambda x: f"{x:,}")
    return table

## 1. Read Data

In [10]:
path_to_train = "train_data.csv"
df = pd.read_csv(path_to_train)
df["review_length"] = df["reviewText"].str.len()

path_to_test = "test_data.csv"
test = pd.read_csv(path_to_test)
test["review_length"] = test["reviewText"].str.len()

In [11]:
overview(df)

  Number of Rows  = 838,944
Number of Columns = 12


,Missing Percentage,Missing Count,Data Type
Column,,,
vote,77.2,"647,476",object
style,41.5,"348,331",object
reviewerName,0.0,227,object
summary,0.0,76,object
reviewTime,0.0,0,object
reviewerID,0.0,0,object
asin,0.0,0,object
reviewText,0.0,0,object
overall,0.0,0,int64


## 2. Basic Preprocess

### 2.1. Drop Entirely Duplicated Rows

In [12]:
df.duplicated().sum()

np.int64(8327)

In [13]:
df = df.drop_duplicates()

### 2.2. Drop Useless Columns

In [14]:
df = df.drop(columns = ["style" , "reviewTime" , "unixReviewTime",
                        "reviewerID" , "reviewerName" , "vote"])
test = test.drop(columns = ["style" , "reviewTime" , "unixReviewTime",
                        "reviewerID" , "asin" , "reviewerName" , "vote"])

### 2.3. Drop Null, based on `summary`

In [15]:
df = df.dropna(subset = ["summary"])

### 2.4. Drop Outliers Based on `review_length`

In [16]:
outlier_boundary = 2_000
df = df.loc[df["review_length"] <= outlier_boundary]

### 2.5. Rename Columns

In [17]:
rename_dict = {
    "overall" : "y",
    "reviewText" : "text",
    "review_length" : "len",
}
df = df.rename(columns = rename_dict)
test = test.rename(columns = rename_dict)

### 2.6. Overview Train Data

In [18]:
overview(df)

  Number of Rows  = 799,898
Number of Columns = 6


,Missing Percentage,Missing Count,Data Type
Column,,,
asin,0.0,0,object
text,0.0,0,object
summary,0.0,0,object
y,0.0,0,int64
len,0.0,0,int64
verified,0.0,0,bool


In [19]:
data = df["len"]
print(f"minimum = {data.min()}")
print(f"median  = {int(data.median())}")
print(f"maximum = {data.max()}")
print(f"mean    = {int(data.mean())}")
print(f"std     = {data.std():.1f}")

minimum = 200
median  = 396
maximum = 2000
mean    = 527
std     = 357.8


In [20]:
df.head()

,y,verified,asin,text,summary,len
0,2,False,0511189877,I have an older URC-WR7 remote and thought thi...,Cannot Learn,561
1,5,True,0511189877,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,450
2,4,True,0511189877,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,205
3,5,True,0511189877,I got tired of the remote being on the wrong s...,Same as TWC remote,284
4,5,True,0594459451,After purchasing cheap cords from another webs...,Good Quality Cord,247


### 2.7. Overview Test Data

In [21]:
overview(test)

  Number of Rows  = 20,000
Number of Columns = 4


,Missing Percentage,Missing Count,Data Type
Column,,,
summary,0.0,2,object
text,0.0,0,object
len,0.0,0,int64
verified,0.0,0,bool


In [22]:
test.head()

,verified,text,summary,len
0,True,The name and description of this device are mi...,The prize for most useless invention of all ti...,1115
1,True,One of the molex connectors on the power suppl...,MELTED MOLEX CONNECTOR,231
2,True,Remote constantly disconnects/ Roku player fre...,Bricked on the regular,225
3,True,I purchased this 4 year protection plan for a ...,DO NOT BUY!!!,1624
4,True,I bought one of these and have regretted it ev...,Nightmare - don't buy,2829


In [23]:
data = test["len"]
print(f"minimum = {data.min()}")
print(f"median  = {int(data.median())}")
print(f"maximum = {data.max()}")
print(f"mean    = {int(data.mean())}")
print(f"std     = {data.std():.1f}")

minimum = 200
median  = 424
maximum = 17583
mean    = 647
std     = 692.8


### 2.8. Optional: Drop `review_length`

In [25]:
# df = df.drop(columns = ["len"])
# test = test.drop(columns = ["len"])

### 2.9. Optional: Drop `summary`

In [26]:
# df = df.drop(columns = ["summary"])
# test = test.drop(columns = ["summary"])

### 2.10. Optional: Drop `verified`

In [27]:
df = df.drop(columns = ["verified"])
test = test.drop(columns = ["verified"])

### 2.11. Fill `summary` in Test Data

In [28]:
test["summary"] = test["summary"].fillna("")

## 3. Advanced Preprocess

### 3.1. Lowercase everything

In [29]:
def lowercase(col):
    return col.str.lower()

In [30]:
df["text"] = lowercase(df["text"])
test["text"] = lowercase(test["text"])

# You might need to comment out the next 2 lines    
df["summary"] = lowercase(df["summary"])
test["summary"] = lowercase(test["summary"])

### 3.2. Handle Contractions

In [31]:
import re
def expand_contractions(col):
    pattern = re.compile(r"\b(" + "|".join(contraction_map.keys()) + r")\b")
    return col.apply(lambda x: pattern.sub(lambda m: contraction_map[m.group(0)] , x))

In [33]:
contraction_map = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have",
    "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",
    "didn't": "did not", "doesn't": "does not", "don't": "do not",
    "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not",
    "haven't": "have not", "he'd": "he would", "he'd've": "he would have",
    "he'll": "he will", "he'll've": "he will have", "he's": "he is",
    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
    "how's": "how is", "i'd": "i would", "i'd've": "i would have",
    "i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have",
    "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
    "it'll": "it will", "it'll've": "it will have", "it's": "it is",
    "let's": "let us", "ma'am": "madam", "might've": "might have",
    "mightn't": "might not", "mightn't've": "might not have", "must've": "must have",
    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
    "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not",
    "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have",
    "she'll": "she will", "she'll've": "she will have", "she's": "she is",
    "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
    "so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have",
    "that's": "that is", "there'd": "there would", "there'd've": "there would have",
    "there's": "there is", "they'd": "they would", "they'd've": "they would have",
    "they'll": "they will", "they'll've": "they will have", "they're": "they are",
    "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
    "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
    "what'll've": "what will have", "what're": "what are", "what's": "what is",
    "what've": "what have", "when's": "when is", "when've": "when have",
    "where'd": "where did", "where's": "where is", "where've": "where have",
    "who'll": "who will", "who'll've": "who will have", "who's": "who is",
    "who've": "who have", "why's": "why is", "why've": "why have",
    "will've": "will have", "won't": "will not", "won't've": "will not have",
    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
    "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
    "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
    "you're": "you are", "you've": "you have"
}

In [35]:
len(contraction_map)

115

In [36]:
df["text"] = expand_contractions(df["text"])
test["text"] = expand_contractions(test["text"])

# You might need to comment out the next 2 lines    
df["summary"] = expand_contractions(df["summary"])
test["summary"] = expand_contractions(test["summary"])

## 3.4. Remove HTML Tags

In [53]:
df.loc[df["text"].str.contains("<") , "text"].shape

(13999,)

In [47]:
df.loc[df["summary"].str.contains("<") , "text"].shape

(125,)

In [54]:
str_html2 = df.loc[df["text"].str.contains(r"<.*?>") , "text"].iloc[25]
str_html22 = re.sub(r"<.*?>", "", str_html2)
print(textwrap.fill(str_html2 , width = 70))
print(70 * "-")
print(textwrap.fill(str_html22 , width = 70))

this filter produces pinkish glow. other reviewers have mentioned
this. do not buy this. wasted my money. <a data-hook="product-link-
linked" class="a-link-normal" href="/tiffen-58mm-uv-protection-
filter/dp/b00004zcji/ref=cm_cr_arp_d_rvw_txt?ie=utf8">tiffen 58mm uv
protection filter</a>
----------------------------------------------------------------------
this filter produces pinkish glow. other reviewers have mentioned
this. do not buy this. wasted my money. tiffen 58mm uv protection
filter


In [50]:
str_html1 = df.loc[df["text"].str.contains(r"<.*?>") , "text"].iloc[5]
str_html11 = re.sub(r"<.*?>", "", str_html1)
print(textwrap.fill(str_html1 , width = 70))
print(70 * "-")
print(textwrap.fill(str_html11 , width = 70))

simply wonderful!  i am very happy with these headphones.  in addition
to sounding great, they are comfortable to wear for long periods.  i
am a long-time fan of sennheiser, since listening to their "open air"
headphones back around 1972.  <a data-hook="product-link-linked"
class="a-link-normal" href="/sennheiser-hd-600-open-back-professional-
headphone/dp/b00004sy4h/ref=cm_cr_arp_d_rvw_txt?ie=utf8">sennheiser hd
600 open back professional headphone</a>
----------------------------------------------------------------------
simply wonderful!  i am very happy with these headphones.  in addition
to sounding great, they are comfortable to wear for long periods.  i
am a long-time fan of sennheiser, since listening to their "open air"
headphones back around 1972.  sennheiser hd 600 open back professional
headphone


In [55]:
df["text"] = df["text"].apply(lambda x: re.sub(r"<.*?>", "", x))
df["summary"] = df["summary"].apply(lambda x: re.sub(r"<.*?>", "", x))

test["text"] = test["text"].apply(lambda x: re.sub(r"<.*?>", "", x))
test["summary"] = test["summary"].apply(lambda x: re.sub(r"<.*?>", "", x))

### 3.3. Clean Spaces

In [56]:
def clean_spaces(col):
    return col.str.replace(r"\s+" , " " , regex = True).str.strip()

In [57]:
df["text"] = clean_spaces(df["text"])
test["text"] = clean_spaces(test["text"])

# You might need to comment out the next 2 lines
df["summary"] = clean_spaces(df["summary"])
test["summary"] = clean_spaces(test["summary"])

## 4. Save Cleaned Data

In [58]:
path_to_clean_train = "train_data_cleaned.csv"
path_to_clean_test = "test_data_cleaned.csv"

In [59]:
df.to_csv(path_to_clean_train)
test.to_csv(path_to_clean_test)